# Part 2.4. Drop Out

In [0]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# 1. Overfitting과 Underfitting

![11-1.png](./img/11-1.png)

오버피팅이란 해당 데이터에 너무 과하게 피팅이 된 경우를 말하고, 언더피팅은 너무 피팅이 안 되서 경향성을 제대로 파악하지 못한 경우를 말한다. 오버피팅이 되면 학습 데이터에는 높은 성능을 보이지만 테스트 데이터에는 낮은 성능을 보인다.

![11-2.png](./img/11-2.png)

학습 데이터에 오버피팅이 되버리면 새로운 데이터를 받아들였을 때 오버피팅이 덜 된 모델보다는 더 많은 에러를 발생시킨다. 

## 2. Drop Out

![11-3.png](./img/11-3.png)

학습을 진행하면서 각 layer에 존재하는 노드들을 무작위로 설정된 비율에 따라서 껐다 켰다를 반복하는 방법이다. 사용한다 결정된 노드들만을 가지고 학습을 한다. 이 때 반복 학습 시 매 iteration마다 사용되는 노드는 다르다.

## 3. MNIST Example

In [0]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [0]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3         # 드롭아웃 비율 784 개 중 약 548개만 사용 

In [0]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [0]:
# nn layers
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [18]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [0]:
# model
model = torch.nn.Sequential(linear1, relu, dropout,     # 매 layer에서 drop out
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

In [0]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
# train
total_batch = len(data_loader)
model.train()    # 모델을 학습하는 데 사용하겠다 설정 그러므로 dropout은 True
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.309896529
Epoch: 0002 cost = 0.143717140
Epoch: 0003 cost = 0.113897748
Epoch: 0004 cost = 0.095177643
Epoch: 0005 cost = 0.085126437
Epoch: 0006 cost = 0.072515391
Epoch: 0007 cost = 0.066215232
Epoch: 0008 cost = 0.065225162
Epoch: 0009 cost = 0.060729682
Epoch: 0010 cost = 0.054089081
Epoch: 0011 cost = 0.050674122
Epoch: 0012 cost = 0.048585333
Epoch: 0013 cost = 0.048006427
Epoch: 0014 cost = 0.044697393
Epoch: 0015 cost = 0.043174274
Learning finished


In [22]:
# test
with torch.no_grad():
    model.eval()    # 모델을 평가하는 데 사용하겠다 설정 그러므로 dropout은 False

    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9767999649047852


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
